# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
from azureml.core import Workspace, Experiment

subscription_id = 'cdbe0b43-92a0-4715-838a-f2648cc7ad21'
resource_group = 'aml-quickstarts-141070'
workspace_name = 'quick-starts-ws-141070'

ws = Workspace(subscription_id, resource_group, workspace_name)

# choose a name for experiment
experiment_name = 'lab3-experiment'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141070
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-141070


In [9]:
for ds in ws.datasets:
    print(ds)

creditcard


In [10]:
dataset = ws.datasets["creditcard"]
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/03-21-2021_065225_UTC/creditcard.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "caf0c66f-ea6b-4ab6-b3b7-877333c6b7a2",
    "name": "creditcard",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-141070', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-141070')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute

# the code sninppet is based on official Microsoft documentation, which can be found
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python
from azureml.core.compute_target import ComputeTargetException

# A name for the cluster
cpu_cluster_name = "lab3-cluster"

# Verification that cluster does not exist yet
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('The cluster already exists')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

The cluster already exists
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric": "accuracy"
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="classification",
    training_data=dataset,
    label_column_name="Class",
    compute_target=cpu_cluster,
    blocked_models=["XGBoostClassifier"],
    **automl_settings)

In [13]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
import joblib

best_automl_run, best_automl_model = remote_run.get_output()

#print('Best Run Id: ', best_automl_run.id)


In [17]:
os.makedirs('./outputs', exist_ok=True)

joblib.dump(best_automl_model, filename='outputs/automl.joblib')

model_name = best_automl_run.properties['model_name']
model_name

'AutoML1b4d8042d85'

In [18]:
# register model in workspace
model = remote_run.register_model(model_name=model_name, description="Card Fraud Detection", tags=None)
remote_run.model_id # Use this id to deploy the model as a web service in Azure

'AutoML1b4d8042d85'

In [19]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_automl_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['0', '51', '79', '78', '52'],
 'weights': [0.4444444444444444,
             0.2222222222222222,
             0.1111111111111111,
             0.1111111111111111,
             0.1111111111111111]}

0 - maxabsscaler
{'copy': True}

0 - lightgbmclassifier
{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': 1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': T

In [20]:
print(best_automl_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                num_leaves=140,
                                                                                                objective=None,
                                                                                                random_state=None,
                                                     

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [32]:
%run score.py

Exception: Error, output schema already defined for function: __main__.run.

In [22]:
pip install inference-schema

Note: you may need to restart the kernel to use updated packages.


In [35]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

env = best_automl_run.get_environment()
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py', environment=env)

service_name = 'fraud-detection-1'

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws, name=service_name, models=[model], inference_config=inference_config, 
                       deployment_config=aci_config, overwrite=True)
service.wait_for_deployment(show_output=True)

print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-21 21:46:42+00:00 Creating Container Registry if not exists.
2021-03-21 21:46:42+00:00 Registering the environment.
2021-03-21 21:46:42+00:00 Use the existing image.
2021-03-21 21:46:43+00:00 Generating deployment configuration.
2021-03-21 21:46:44+00:00 Submitting deployment to compute..
2021-03-21 21:46:49+00:00 Checking the status of deployment fraud-detection-1..
2021-03-21 21:50:37+00:00 Checking the status of inference endpoint fraud-detection-1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [36]:
print("Scoring URI: " + service.scoring_uri)

Scoring URI: http://9792dabd-c7a6-4fce-b938-06d8513da0e3.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service